In [69]:
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen

In [70]:
url = "https://www.ibge.gov.br/indicadores.html"


In [71]:
client = urlopen(url)
page_html = client.read()
client.close()

In [119]:
page_soup = soup(page_html)


In [120]:
span_indicadores = page_soup.findAll('span', {"class": "nonsprite"})

In [121]:
indicadores = []
for indicador in span_indicadores:
    #Adiciona o titulo dos indicadores e substitui caracteres
    indicadores.append(indicador
    .text
    .replace("\r", "")
    .replace("\n", "")
    .replace(" ", ""))

#Adiciona espaço entre indicador e sua unidade
for i in range(len(indicadores)):
    indice = indicadores[i].find("(")
    indicadores[i] = indicadores[i][:indice] + " " + indicadores[i][indice:]


In [122]:
td_ultimo_valor = page_soup.findAll('td', {"class": "ultimo"})

ultimo_valor = []
data_ultimo = []

for item in td_ultimo_valor:
    #Adicionando ultima data
    data_ultimo.append(item.small.text)

    #Adicionando ultimo indice e deixando apenas o valor
    ultimo_valor.append(item
    .text
    .replace("\r", "")
    .replace("\n", "")
    .replace("Último", "")
    .replace(data_ultimo[-1], "")
    .replace(" ", "")
    .replace(",", "."))

#Convertendo string em float
for i in range(len(ultimo_valor)):
    ultimo_valor[i] = str(ultimo_valor[i])
    # Verifica se a string contém apenas um ponto decimal
    if ultimo_valor[i].count('.') == 1:
    # Converte a string para float
        ultimo_valor[i] = float(ultimo_valor[i])
    else:
        ultimo_valor[i] = float(ultimo_valor[i].replace(".", ""))

In [123]:
td_anterior = page_soup.findAll('td', {"class": "desktop-tablet-only anterior"})

anterior_valor = []
data_anterior = []

for item in td_anterior:
    #Adiciona data anterior
    data_anterior.append(item.small.text)

    #Adiciona indice anterior e deixa apenas o valor
    anterior_valor.append(item
    .text
    .replace("\r", "")
    .replace("\n", "")
    .replace("Anterior", "")
    .replace(data_anterior[-1], "")
    .replace(" ", "")
    .replace(",", "."))

#Convertendo string em float
for i in range(len(ultimo_valor)):
    anterior_valor[i] = str(anterior_valor[i])
    # Verifica se a string contém apenas um ponto decimal
    if anterior_valor[i].count('.') == 1:
    # Converte a string para float
        anterior_valor[i] = float(anterior_valor[i])
    else:
        anterior_valor[i] = float(anterior_valor[i].replace(".", ""))

In [124]:
pd.set_option("display.float_format", '{:.2f}'.format)
df = pd.DataFrame(list(zip(
    indicadores, ultimo_valor, data_ultimo, anterior_valor, data_anterior)),
    columns =['Indicador', 'Ultimo-Valor', 'Data-Ultimo-Valor', 'Valor-Anterior', 'Data-Valor-Anterior']
    )
df

,Indicador,Ultimo-Valor,Data-Ultimo-Valor,Valor-Anterior,Data-Valor-Anterior
0,IPCA (%),0.26,set 2023,0.23,ago 2023
1,INPC (%),0.11,set 2023,0.20,ago 2023
2,IPCA-15 (%),0.21,out 2023,0.35,set 2023
3,IPP (%),1.11,set 2023,0.75,ago 2023
4,Custodom² (%),0.02,set 2023,0.18,ago 2023
5,VariaçãodoPIB (%),3.40,2º tri 2023,4.00,1º tri 2023
6,PIBpercapita (R$),3593569.00,2020,3516170.00,2019
7,Indústria (%),0.40,ago 2023,-0.60,jul 2023
8,Comércio (%),-0.20,ago 2023,0.70,jul 2023
9,Serviços (%),-0.90,ago 2023,0.40,jul 2023


In [125]:
#Não foi realizada uma manipulação de data por estarem em períodos diferentes
df.dtypes

Indicador               object
Ultimo-Valor           float64
Data-Ultimo-Valor       object
Valor-Anterior         float64
Data-Valor-Anterior     object
dtype: object

In [129]:
#df.to_csv("./indices_ibge.csv", index=False, sep=';')